In [1]:
import numpy as np
import os 
os.environ['TF_ENABLE_ONEDNN_OPTS']='0'
import tensorflow as tf 
import tensorflow_datasets as tfds
from tensorflow import keras
import matplotlib.pyplot as plt

In [2]:
ALL_DATA_DIR = '/media/windows/'
training = ALL_DATA_DIR + 'training/'
validation = ALL_DATA_DIR + 'validation/'
scaled = ALL_DATA_DIR + 'scaled/'
scaled_wo_rotate = ALL_DATA_DIR + "scaled_wo_rotate/"

scaled_wo_rotate_training = scaled_wo_rotate + "training/"
scaled_wo_rotate_validation = scaled_wo_rotate + "validation/"

datFile = training + "shape_predictor_68_face_landmarks.dat"

In [3]:
batch_size = 32 
img_height = 224
img_width = 224 

In [4]:
validation_s = tf.keras.utils.image_dataset_from_directory(scaled_wo_rotate_validation, labels = 'inferred',
            image_size=(img_height,img_width), batch_size=batch_size, seed=42).prefetch(1)

Found 50947 files belonging to 7 classes.


2022-09-01 13:59:44.799725: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-01 13:59:44.808636: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-01 13:59:44.811270: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-01 13:59:44.814334: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the approp

In [5]:
training_s = tf.keras.utils.image_dataset_from_directory(scaled_wo_rotate_training, labels = 'inferred',
            image_size=(img_height,img_width), batch_size=batch_size, seed=42).prefetch(1)

Found 399839 files belonging to 7 classes.


In [6]:
base_model = keras.applications.MobileNetV3Small(input_shape=(224,224,3), include_top = False, weights='imagenet')

In [7]:
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(7, activation='softmax')(avg)
model = keras.Model(inputs=base_model.input, outputs=output)

In [8]:
for layer in base_model.layers:
    layer.trainable = False

In [9]:
model.compile(optimizer="Adam",
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
model.fit(training_s, validation_data=validation_s, epochs=3)

Epoch 1/3


2022-09-01 14:00:29.633024: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8400


12495/12495 [==============================] - 420s 33ms/step - loss: 1.5012 - accuracy: 0.4265 - val_loss: 1.4194 - val_accuracy: 0.4648
Epoch 2/3
12495/12495 [==============================] - 411s 33ms/step - loss: 1.4168 - accuracy: 0.4606 - val_loss: 1.4086 - val_accuracy: 0.4637
Epoch 3/3
12495/12495 [==============================] - 415s 33ms/step - loss: 1.4066 - accuracy: 0.4651 - val_loss: 1.3992 - val_accuracy: 0.4696


In [11]:
for layer in base_model.layers:
    layer.trainable = True

In [12]:
opt = keras.optimizers.Adam(learning_rate=1e-4) #0.0001

In [13]:
es = keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)

In [14]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
history = model.fit(training_s, validation_data=validation_s, epochs=20, callbacks=[es])
model.save('0901_mobilenet.h5')

Epoch 1/20
12495/12495 [==============================] - 813s 65ms/step - loss: 0.9363 - accuracy: 0.6510 - val_loss: 0.7765 - val_accuracy: 0.7136
Epoch 2/20
12495/12495 [==============================] - 816s 65ms/step - loss: 0.7279 - accuracy: 0.7284 - val_loss: 0.7107 - val_accuracy: 0.7395
Epoch 3/20
12495/12495 [==============================] - 799s 64ms/step - loss: 0.6415 - accuracy: 0.7631 - val_loss: 0.6707 - val_accuracy: 0.7564
Epoch 4/20
12495/12495 [==============================] - 810s 65ms/step - loss: 0.5775 - accuracy: 0.7870 - val_loss: 0.6469 - val_accuracy: 0.7686
Epoch 5/20
12495/12495 [==============================] - 801s 64ms/step - loss: 0.5266 - accuracy: 0.8070 - val_loss: 0.6359 - val_accuracy: 0.7731
Epoch 6/20
12495/12495 [==============================] - 810s 65ms/step - loss: 0.4838 - accuracy: 0.8230 - val_loss: 0.6497 - val_accuracy: 0.7718
Epoch 7/20
12495/12495 [==============================] - 809s 65ms/step - loss: 0.4472 - accuracy: 0.8362

In [16]:
model.evaluate(training_s)

12495/12495 [==============================] - 377s 30ms/step - loss: 0.4756 - accuracy: 0.8260


[0.4756191074848175, 0.8260249495506287]